# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install .`

In [1]:
# %pip install ..

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pd.read_csv(demonstration_fp)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1922-03-23,14:57:38,2022-08-13 08:42:37
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1938-03-22,17:16:30,2022-07-24 06:45:36
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1909-11-06,15:57:59,2022-07-21 02:32:58
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1915-10-14,14:17:36,2022-07-19 13:22:18
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1929-04-18,13:23:39,2022-07-26 11:47:42


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [4]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv(demonstration_fp, dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1922-03-23,14:57:38,2022-08-13 08:42:37
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1938-03-22,17:16:30,2022-07-24 06:45:36
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1909-11-06,15:57:59,2022-07-21 02:32:58
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1915-10-14,14:17:36,2022-07-19 13:22:18
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1929-04-18,13:23:39,2022-07-26 11:47:42


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [5]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [6]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [7]:
df["Birthday"][0]

datetime.date(1922, 3, 23)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [8]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:126: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [9]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [10]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [11]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,87,"B4L#w<^1N*VUB\R7 2nG"" 28[D ~U",female,2,0,6.833824,BE[T*P^Pp>[ 6C,C,1921-03-29,12:43:03,2022-07-15 22:50:45
1,253,5:p&GbGxbG%3;^<~l%1 7}: 9#&M;}0i%h[8D\&se5XoYq...,male,6,2,11.221334,G?8.,S,1926-05-27,16:55:15,2022-08-13 09:51:28
2,303,"C,hHIp@2 Hd3K2D*.x!T4. m(RM)J,sE4\-n7p9jy3<E~...",male,20,0,2.057991,C4M+aup,S,1908-09-17,16:13:17,2022-07-19 01:19:23
3,333,"58{&r;+zm`_.bMS,)RBWA^Si4t*Y(",male,<NA>,0,232.294111,CARO%.9~,S,1921-11-17,13:46:05,2022-07-19 14:04:54
4,19,"{$ii""pf;<q'%""} Qn'$|n8:EpTt*CT]+\b~# `KOq+__z{...",male,45,0,17.709321,A;hu'rn1l? ^D[,S,1925-10-10,10:55:38,2022-07-19 10:21:41


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"DghYoa=sz 89y -Fewr;7d+F8Wv~*m,@NHNs)rWXFg(>5...",female,72,2,11.012113,<NA>,S,1934-11-29,16:58:15,2022-08-10 20:09:20
1,2,x]B1svv#%Ba2>W!&:;Y<=YScpB$d_z8L/ot[!m,male,13,0,44.211592,<NA>,C,1935-09-16,14:22:55,2022-07-26 19:08:15
2,3,"""`5b(0?oJRvlfiHduQ%MHA-y5 .CG"" ?z9+4I;;qle@Vp~M*T",female,79,2,201.535015,A>CA,S,1939-10-01,17:18:28,2022-08-15 00:05:43
3,4,#%3 2]d&}pH(dvr&{RiJ7 CJYEYF-$hZx/q-SFPkS>nurN...,male,48,1,25.199516,F\*,S,1920-02-04,12:48:31,2022-07-26 05:01:19
4,5,"ZM l9V }(Pn; wc V !Qe f""<60:-6 H kE8\Q T2>9</F...",male,32,0,14.286797,EP {1,S,1908-07-19,15:14:03,2022-07-26 22:41:16


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Elizabeth Sandoval,male,65,0,76.620037,<NA>,Q,1922-03-02,14:47:02,2022-08-05 21:22:06
1,2,Nicole Parker,male,<NA>,0,21.391347,<NA>,S,1940-03-05,14:19:56,2022-07-28 00:40:04
2,3,Linda Hardy,male,51,1,49.207215,"GTM@d,d",Q,1935-04-03,11:21:03,2022-08-14 04:27:38
3,4,Susan Ray,male,22,1,52.878771,B&3wVwzQeU,S,1903-10-25,10:59:57,2022-07-20 17:12:08
4,5,Carl Jones,female,51,0,177.021382,<NA>,S,1904-12-27,18:11:16,2022-08-09 10:42:04


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Ariel Clarke,male,<NA>,0,0.047460,<NA>,S,1938-08-07,11:27:23,2022-08-02 20:36:10
1,2,Casey Bullock,male,37,0,1.828377,<NA>,Q,1917-11-10,16:41:05,2022-07-20 08:24:47
2,3,Andrea Delacruz,male,35,0,1.135577,EGHWG-V/,S,1934-06-06,12:37:41,2022-08-01 14:09:06
3,4,Sarah Black,male,28,1,1.167487,<NA>,S,1906-12-02,11:35:10,2022-08-01 03:51:53
4,5,Kathy Murray,female,<NA>,0,0.282576,<NA>,S,1921-02-13,17:29:30,2022-07-29 04:12:57


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Craig Reyes,female,<NA>,0,1.934005,A194,S,1916-05-11,13:46:23,2022-08-07 02:10:02
1,2,Frank Adams,male,28,0,0.244927,<NA>,S,1905-10-29,14:16:26,2022-07-29 14:58:19
2,3,Bruce Mullen,male,25,0,0.290633,<NA>,S,1930-01-22,15:50:25,2022-07-23 07:12:48
3,4,Nathan Barton,male,32,1,6.161548,<NA>,S,1907-12-18,16:21:42,2022-07-18 04:17:26
4,5,Mark Jimenez,male,20,0,1.506060,<NA>,S,1937-02-23,12:54:48,2022-08-02 10:22:36
5,6,Terry Mason,male,35,0,3.225576,<NA>,S,1904-12-03,17:55:37,2022-07-22 23:29:39
6,7,Elizabeth Allen,male,<NA>,2,1.965927,<NA>,Q,1916-10-13,13:38:14,2022-08-10 09:42:55
7,8,Kim Madden,female,<NA>,2,1.054873,C70,Q,1916-02-14,17:40:14,2022-08-11 12:41:00
8,9,Jerry Norris,female,23,0,0.757025,<NA>,S,1905-08-16,17:39:27,2022-07-26 14:51:10
9,10,Amber Chan,male,28,0,0.257668,<NA>,S,1917-08-17,14:11:21,2022-08-06 21:27:56
